In [1]:
import os
import sys
sys.path.append('..')
from backtest.operators import * 
from data_center.tej_handler import TEJHandler
from monitor.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import tejapi

import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
# TEJHandler()

### Get Weighting

##### 新增過去幾天參數

In [2]:
collection = "monthly_revenue"
monthly_rev_pmart = TWMarketMonitor().directly_get_pmart(collection, 39000)

In [14]:
monthly_rev_pmart['revenue_yoy'] = monthly_rev_pmart['單月營收(千元)'] / monthly_rev_pmart['去年單月營收(千元)'] -1
example_factor = ts_rank(monthly_rev_pmart['revenue_yoy'],252)
example_factor

symbol,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,1110,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def get_weighting(factor):
    demean = factor.sub(factor.mean(axis=1), axis=0)
    weighting = demean.div(demean.abs().sum(axis=1), axis=0)
    return weighting

### 假設出訊號後隔天收盤進場

In [12]:
example_signal = get_weighting(example_factor)
example_weighting = example_signal.shift(2) #隔天收盤進場，後天才有部位

### Get returns

In [7]:
collection = 'securities_returns'
returns_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [8]:
collection = 'securities_trading_data'
ohlcv_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [12]:
ohlcv_pmart['開盤價']

symbol,0050,0051,0052,0053,0055,0056,0057,0061,006201,006203,...,9946,9949,9950,9951,9955,9957,9958,9960,9962,T1012Y
datetime,,,,,,,,,,,,,,,,,,,,,
2024-03-29,NaN,NaN,158.45,NaN,NaN,39.75,116.85,16.92,21.96,75.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.90,NaN
2024-04-01,158.20,78.7,160.00,85.95,25.38,39.89,117.85,16.90,22.25,75.90,...,25.50,18.6,17.00,72.4,25.15,6.44,220.0,27.1,18.10,NaN
2024-04-02,158.45,78.9,159.25,86.80,25.32,39.62,117.50,17.19,22.25,75.50,...,26.25,18.8,17.50,72.7,25.60,6.40,218.5,27.0,18.15,NaN
2024-04-03,158.65,78.8,161.20,85.70,25.25,39.69,117.25,17.23,22.18,75.75,...,26.70,18.8,17.30,72.6,26.00,6.38,213.5,26.9,18.30,NaN
2024-04-08,158.65,78.8,161.00,86.95,25.03,39.77,117.45,17.04,22.27,75.95,...,26.45,19.1,17.35,72.5,25.55,6.84,216.0,26.9,18.15,63.5


In [11]:
example_returns = returns_pmart['日報酬率(%)']*0.01
example_returns

symbol,0015,0029,0050,0051,0052,0053,0055,0056,0057,0061,...,IX0059,IX0060,IX0075,IX0085,IX0099,IX0118,MSCI,T1001Y,T1010Y,T1012Y
datetime,,,,,,,,,,,,,,,,,,,,,
2024-04-01,0.0,-0.009422,-0.003167,0.004456,-0.003139,-0.006407,-0.002364,-0.006774,-0.005991,0.017762,...,0.000191,NaN,-0.001288,NaN,0.025392,NaN,NaN,0.0,-0.009422,0.0
2024-04-02,0.0,0.012581,0.014295,0.000000,0.020151,0.018757,-0.002765,0.007325,0.013775,-0.003490,...,-0.001669,0.004918,0.016493,0.006206,0.018698,0.002900,0.016478,0.0,0.012581,0.0
2024-04-03,0.0,0.030303,-0.006890,-0.001267,-0.007716,-0.002877,-0.008713,-0.003260,-0.004246,-0.005254,...,-0.001528,0.001224,-0.005212,-0.002409,0.015316,0.000537,-0.009002,0.0,0.030303,0.0
2024-04-08,0.0,-0.007941,0.005676,0.004442,0.002177,0.001731,0.007990,0.004277,0.004691,-0.001174,...,0.003253,0.001955,-0.029432,-0.002318,0.016824,0.002674,0.003356,0.0,-0.007941,0.0


### 將weighting 與 returns結合獲得績效

In [10]:
stocks = 20
top = example_weighting.apply(lambda row: row.nlargest(stocks), axis=1).fillna(0)
example_equalweight = top.applymap(lambda x: 1/stocks if x != 0 else 0)
recent_w_weighting = example_equalweight#[-5:].dropna(axis =1, how = 'all')
recent_w_weighting = recent_w_weighting.loc[:, (recent_w_weighting != 0.0).any(axis=0)]
recent_w_weighting

/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_16458/681453456.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  example_equalweight = top.applymap(lambda x: 1/stocks if x != 0 else 0)


symbol
datetime
1990-01-04
1990-01-05
1990-01-06
1990-01-08
1990-01-09
...
2024-03-29
2024-04-01
2024-04-02


In [15]:
recent_w_returns = example_returns * recent_w_weighting
recent_w_profit = recent_w_returns.dropna(axis =1, how='all').dropna(axis=0, how='all')
recent_w_profit

symbol
datetime


In [16]:
recent_w_profit.sum(axis =1)#.sum()

Series([], dtype: float64)